In [1]:
from testing import PSGANInference
from psgan import PSGAN
from models.target import YOLOv8TargetModel
# Configuration
IMG_SIZE = 256
PATCH_SIZE = 32
CHECKPOINT_DIR = './checkpoints_distributed/'
MAX_EPOCHS = 100

# Roboflow dataset configuration
DATASET_CONFIG = {
    'api_key': "Yp0GigDoIjDjIoTrGiyA",
    'workspace': "mrsyaban",
    'project': "traffic-signs-id",
    'version': 2
}

# YOLO model path
YOLO_MODEL_PATH = './pretrained_models/yolov8x-best.pt'

# Initialize YOLO model
print("Loading YOLO model...")
yolo_model = YOLOv8TargetModel(
    model_path=YOLO_MODEL_PATH,
    confidence_threshold=0.1
)

# Initialize inference model
print("Initializing PSGAN inference...")
psgan_inference = PSGANInference(
    checkpoint_dir=CHECKPOINT_DIR,
    patch_size=PATCH_SIZE,
    image_size=IMG_SIZE
)

# Evaluate across epochs
print("Starting multi-epoch evaluation...")
results = psgan_inference.test_against_yolo_epochs(
    dataset_path=None,  # Will be auto-downloaded
    dataset_config=DATASET_CONFIG,
    yolo_model=yolo_model,
    max_epochs=MAX_EPOCHS,
    max_batches=200,  # Limit for faster evaluation
    save_results=True
)

# Visualize results
if results:
    print("Creating mAP decline visualization...")
    psgan_inference.visualize_map_decline(
        results=results,
        save_path='map_decline_across_epochs.png',
        title='mAP Decline Across PS-GAN Training Epochs'
    )

    # Visualize adversarial samples from latest epoch
    print("Creating adversarial samples visualization...")
    psgan_inference.visualize_latest_adversarial_samples(
        yolo_model=yolo_model,
        num_samples=6,
        save_path='latest_epoch_adversarial_samples.png'
    )

    # Print summary
    print("\n=== Evaluation Summary ===")
    print(f"Dataset: {results['dataset_info']['num_classes']} classes, {results['dataset_info']['num_images']} images")
    print(f"Classes: {results['dataset_info']['classes']}")
    print(f"Baseline mAP: {results['baseline_map']:.4f}")
    print(f"Seed patch mAP: {results['seed_patch_map']:.4f}")
    print(f"Seed patch reduction: {results['baseline_map'] - results['seed_patch_map']:.4f} ({((results['baseline_map'] - results['seed_patch_map'])/results['baseline_map'])*100:.1f}%)")

    if results['map_scores']:
        final_map = results['map_scores'][-1]
        min_map = min(results['map_scores'])
        print(f"Final mAP (last epoch): {final_map:.4f}")
        print(f"Minimum mAP: {min_map:.4f}")
        print(f"Maximum reduction: {results['baseline_map'] - min_map:.4f}")
        print(f"Reduction percentage: {((results['baseline_map'] - min_map)/results['baseline_map'])*100:.1f}%")

        # Show per-epoch details
        print(f"\nPer-epoch results:")
        for epoch in results['epochs'][-5:]:  # Show last 5 epochs
            epoch_data = results['epoch_results'][epoch]
            print(f"Epoch {epoch}: mAP={epoch_data['map']:.4f}")

Calculated layer sizes: 32 -> 16 -> 8 -> 4 -> 2
Figure saved as generated_patches_6.png


In [13]:
!python test_generator.py

Calculated layer sizes: 32 -> 16 -> 8 -> 4 -> 2
Figure saved as generated_patches_250.png
